<a href="https://colab.research.google.com/github/Hlompho-Dash/first_try_of_fastai/blob/master/02_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
------------------------------------------------------------------


In [82]:
print(os.getcwd())

/content/gdrive/MyDrive/first_try_of_fastai


In [83]:
#export

from exp.nb_01 import *

def get_data():
  path2 = "/content/gdrive/MyDrive/first_try_of_fastai/mnist (1).pkl.gz"
  with gzip.open(path2, "rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding = "latin-1")
  return map(tensor, (x_train,y_train,x_valid,y_valid))
def normalize(x, m, s): return (x-m)/s

In [84]:
x_train, y_train, x_valid, y_valid = get_data()

In [85]:
train_mean, train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [86]:
x_train = normalize(x_train, train_mean, train_std)

x_valid = normalize(x_valid, train_mean, train_std)

In [87]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(3.8966e-08), tensor(1.))

In [88]:
#export

def test_near_zero(a, tol=1e-3):  assert a.abs()<tol, f"Near zero: {a}"

In [89]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [90]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

**Basic architecture**

In [91]:
nh = 50

In [92]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh )
b2 = torch.zeros(1)

In [93]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [94]:
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [95]:
def lin(x, w, b): return x@w + b

In [96]:
t = lin(x_valid, w1, b1)
t.mean(),t.std()

(tensor(-0.0803), tensor(0.9851))

In [97]:
def relu(x): return x.clamp_min(0.)

In [98]:
t = relu(lin(x_valid, w1, b1))

In [99]:
t.mean(),t.std()

(tensor(0.3526), tensor(0.5549))

**  FIXING THE VANISHING GRADIENT PROBLEM**

In [100]:
w1 = torch.rand(m,nh)*math.sqrt(2/m)
w1.mean(),w1.std()

(tensor(0.0253), tensor(0.0145))

In [101]:
t= relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.9977), tensor(1.6721))

In [102]:
 #export

 from torch.nn import init

In [103]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode = "fan_out")

t = relu(lin(x_valid, w1, b1))

In [104]:
init.kaiming_normal_??

In [105]:
w1.mean(),w1.std()

(tensor(0.0003), tensor(0.0508))

In [106]:
t.mean(),t.std()

(tensor(0.6425), tensor(0.9018))

In [107]:
w1.shape

torch.Size([784, 50])

In [108]:
import torch.nn

In [109]:
torch.nn.Linear.forward??

In [110]:
torch.nn.functional.linear??

In [111]:
torch.nn.Conv2d??

In [112]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [113]:
def relu(x): return x.clamp_min(0.) - 0.5

In [114]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t.std()

(tensor(0.0595), tensor(0.9018))

In [115]:
def model(xb):
  l1 = lin(xb, w1, b1)
  l2 = relu(l1)
  l3 = lin(l2, w2, b2)

  return l3

In [116]:
%timeit -n 10 _= model(x_valid)

10 loops, best of 3: 15.6 ms per loop


In [117]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

In [118]:
x_valid.shape

torch.Size([10000, 784])

In [119]:
torch.Size([x_valid.shape[0],1])

torch.Size([10000, 1])

In [120]:
model(x_valid).shape

torch.Size([10000, 1])

**LOSS FUNCTION: MSE**

In [121]:
def mse(output, target): return (output.squeeze(-1) - target).pow(2).mean()

In [122]:
y_train, y_valid = y_train.float(), y_valid.float()

In [123]:
preds = model(x_train)

In [124]:
preds.shape

torch.Size([50000, 1])

In [125]:
mse(preds, y_train)

tensor(22.2787)

**GRADIENTS AND BACKWARD PASSES**

In [126]:
def mse_grad(inp, targ):
  # grad of loss with respect to output
  inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1)/ inp.shape[0]

In [127]:
def relu_grad(inp, out):
   # grad of relu with respect to input activations
   inp.g = (inp > 0).float() * out.g

In [128]:
def lin_grad(inp, out, w, b):
  # grad of matmul with respect to input
  inp.g = out.g @ w.t()
  w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
  b.g = out.g.sum(0)

In [129]:
def forward_and_backward(inp, targ):
  # forward pass:
  l1 = inp @ w1 + b1
  l2 = relu(l1)
  out = l2 @ w2 + b2

  loss = mse(out, targ)

  #backward pass:
  mse_grad(out, targ)
  lin_grad(l2, out, w2, b2)
  relu_grad(l1, l2)
  lin_grad(inp, l1, w1, b1)


In [130]:
forward_and_backward(x_train, y_train)

In [131]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [132]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)


In [133]:
def forward(inp, targ):
  # forward
  l1 = inp @ w12 + b12
  l2 = relu(l1)
  out = l2 @ w22 + b22

  return mse(out,targ)

In [134]:
loss = forward(xt2, y_train)

In [135]:
# THIS CELL HAS PROBLEMS, LOOK INTO THEM
"""test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)"""

'test_near(w22.grad, w2g)\ntest_near(b22.grad, b2g)\ntest_near(w12.grad, w1g)\ntest_near(b12.grad, b1g)\ntest_near(xt2.grad, ig)'

BEFORE WE GO INTO REFACTORING THE MODEL NOTE THE FOLLOWING:

1. MATRIX REPRESENTATIONS OF NEURAL NETS. NOT QUITE CLEAR ON THAT
2. THE ".g" OF TENSORS IN THE GRADIENT CALCULATIONS ON THE BACKPROP
3. LEAST THEN LOOK INTO THE SMALL TEST FRAMEWORKS BUILT IN

***REFACTORING THE MODEL***

*LAYERS AS CLASSES*

In [136]:
class Relu():
  def __call__(self, inp):
    self.inp = inp
    self.out = inp.clamp_min(0.) - 0.5
    
    return self.out

  def backward(self): self.inp.g = (self.inp > 0).float() * self.out.g

In [137]:
class Lin():
  def __init__(self,w,b):
    self.w, self.b = w,b
  
  def __call__(self,inp):
    self.inp = inp
    self.out = inp@self.w + self.b
    return self.out

  def backward(self):
    self.inp.g = self.out.g @ self.w.t()
    self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
    self.b.g = self.out.g.sum(0)

In [138]:
class Mse():
  def __call__(self,inp,targ):
    self.inp = inp
    self.targ = targ
    self.out = (inp.squeeze() - targ).pow(2).mean()

  def backward(self):
    self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [139]:
class Model():
  def __init__(self, w1, b1, w2, b2):
    self.layers = [Lin(w1,b1),Relu(), Lin(w2,b2)]
    self.loss = Mse()

  def __call__(self, x , targ):
    for l in self.layers: x = l(x)
    return self.loss(x, targ)

  def backward(self):
    self.loss.backward()
    for l in reversed(self.layers): l.backward()

In [140]:
w1.g,b1.g,w2.g,b2.g = [None]*4

model = Model(w1, b1, w2, b2)

In [141]:
%time loss = model(x_train, y_train)

CPU times: user 85.5 ms, sys: 0 ns, total: 85.5 ms
Wall time: 88.8 ms


In [142]:
%time model.backward()

CPU times: user 4.39 s, sys: 42.5 ms, total: 4.43 s
Wall time: 4.44 s


In [143]:
# initialization and calling, different parameters to be taken
Model(w1, b1, w2, b2)(x_train, y_train)

In [144]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

**Module.forward**

In [145]:
class Module():
  def __call__(self, *args):
    self.args = args
    self.out = self.forward(*args)
    return self.out

  def forward(self): raise Exception("Not implementaed")
  def backward(self): self.bwd(self.out,*self.args)

In [146]:
class Relu(Module):
  def forward(self, inp): return inp.clamp_min(0.) - 0.5
  def bwd(self,out,inp): inp.g = (inp > 0).float() * out.g

In [147]:
class Lin(Module):
  def __init__(self,w,b):
    self.w,self.b = w,b

  def forward(self, inp): return inp@self.w + self.b

  def bwd(self, out, inp):
    inp.g = out.g @ self.w.t()
    self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
    self.b.g = out.g.sum(0)

In [148]:
class Mse(Module):
  def forward(self, inp,targ): return (inp.squeeze() - targ).pow(2).mean()
  def bwd(self,out,inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1)/targ.shape[0]

In [149]:
class Model():
  def __init__(self,w1,b1,w2,b2):
    self.layers = [Lin(w1,b1),Relu(),Lin(w2,b2)]
    self.loss = Mse()

  def __call__(self, x, targ):
    for l in self.layers: x = l(x)
    return self.loss(x,targ)
  
  def backward(self):
    self.loss.backward()
    for l in reversed(self.layers): l.backward()

In [150]:
w1.g,b1.g,w2.g,b2.g = [None]*4

model = Model(w1, b1, w2, b2)

In [151]:
%time loss = model(x_train, y_train)

CPU times: user 82 ms, sys: 0 ns, total: 82 ms
Wall time: 82.3 ms


**WITHOUT EINSUM**

In [152]:
class Lin(Module):
  def __init__(self, w, b): self.w, self.b = w,b

  def forward(self, inp): return inp@self.w + self.b

  def bwd(self, out, inp):
    inp.g = out.g @ self.w.t()
    self.w.g = inp.t() @ out.g
    self.b.g = out.g.sum(0)

In [153]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [154]:
%time loss = model(x_train, y_train)

CPU times: user 89.5 ms, sys: 0 ns, total: 89.5 ms
Wall time: 99.6 ms


In [155]:
%time model.backward()

CPU times: user 204 ms, sys: 1.36 ms, total: 206 ms
Wall time: 207 ms


In [156]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

**nn.Linear**
**nn.Module**

In [157]:
#export
from torch import nn

In [158]:
class Model(nn.Module):
  def __init__(self, n_in, nh, n_out):
    super().__init__()
    self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
    self.loss = mse
  
  def __call__(self, x , targ):
    for l in self.layers: l(x)
    return self.loss(x.squeeze(), targ)

In [159]:
model = Model(m, nh, 1)

In [160]:
%time loss = model(x_train, y_train)

RuntimeError: ignored

In [ ]:
%time loss.backward()

**EXPORT**

In [ ]:
! ./notebook2script.py 02_fully_connected.ipynb